In [1]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from palettable.scientific.sequential import Batlow_5
from palettable.tableau import Tableau_10
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

print(os.environ['SUBGRAPH_URL'])
# logging.basicConfig(level=logging.DEBUG)

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    wide_to_longwide, 
    chart, 
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk


In [2]:
ADDR_BEANSTALK.lower()

'0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5'

In [3]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [10]:
plots = bs.Query.plots(first=100000)
df_plots = sg.query_df([
    plots.pods, 
    plots.farmer.id
])
df_plots = remove_prefix(df_plots, "plots_")

In [11]:
df_plots.pods /= 1e6 
df_plots = df_plots.rename(columns={"farmer_id": "address"})
df_plots = df_plots.groupby("address").sum().reset_index()
df_plots['href'] = df_plots.address.apply(lambda addr: f"https://etherscan.io/address/{addr.lower()}")
df_plots.head()

,address,pods,href
0,0x000000009d3a9e5c7c620514e1f36905c4eb91e6,2.004821e+04,https://etherscan.io/address/0x000000009d3a9e5...
1,0x000000c7000cc57fd1bf6729849b9edcc8fe9102,5.973832e+06,https://etherscan.io/address/0x000000c7000cc57...
2,0x00063ddb30be7bc2292583d5f143e9d6e6228440,2.924322e+04,https://etherscan.io/address/0x00063ddb30be7bc...
3,0x00427c81629cd592aa068b0290425261cbb8eba2,8.278357e+05,https://etherscan.io/address/0x00427c81629cd59...
4,0x0086e622ac7afa3e5502dc895fd0eab8b3a78d97,2.678815e+04,https://etherscan.io/address/0x0086e622ac7afa3...


In [12]:
nrows = 15
radio_sort_dir = alt.binding_radio(name="Sort Direction:", options=['asc', 'desc'])
slider = alt.binding_range(min=1, max=len(df_plots) - nrows, step=1, name='Scroll Offset:')

select_scroll = alt.selection_single(
    name="scroller", fields=['offset'], bind=slider, init={'offset': 1}
)
select_radio_sort_dir = alt.selection_single(
    name="sortdir", fields=["sort_dir"], bind=radio_sort_dir, init={"sort_dir": "desc"}
)

table_base = (
    alt.Chart(df_plots)
    .transform_joinaggregate(rc="count(*)")
    .transform_window(sort=[{"field": "pods"}], frame=[None, 0], sort_field="row_number(*)")
    .transform_calculate(sort_num="sortdir.sort_dir[0] === 'asc' ? datum.sort_field : (datum.rc - datum.sort_field + 1)")
    .transform_fold(['address', 'pods'])
    .transform_filter(
        f"""
        datum.sort_num >= parseInt(scroller.offset)
        && datum.sort_num < (parseInt(scroller.offset) + {nrows})
        """
    )
    .encode(
        x=alt.X(
            "key:N", 
            axis=alt.Axis(
                orient="top", 
                labelAngle=0, 
                title=None, 
                domain=False, 
                ticks=False, 
                labelFontWeight=600, 
                labelFontSize=12
            ), 
        ),
        y=alt.Y("sort_num:O", axis=None), 
    )
) 

table_rect = (
    table_base
    .mark_rect(stroke="black")
    .encode(
        color=alt.condition("datum.sort_num % 2 === 0", alt.value("#e3e3e3"), alt.value("#ffffff")), 
        href="href"
    )
) 
table_text = (
    table_base
    .transform_calculate(
        # label="datum.value"
        label="""
        datum.key === 'address' ? datum.value : 
        datum.key === 'pods' ? format(datum.value, ',d') : 
        datum.value 
        """
    )
    .mark_text()
    .encode(text='label:N')
) 

c = (
    alt.layer(table_rect, table_text)
    .add_selection(select_scroll, select_radio_sort_dir)
    .properties(width=750)
)
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [13]:
output_chart(c)

<IPython.core.display.JSON object>